In [1]:
# Example code for feature extraction using a pre-trained deep learning model (using TensorFlow)
import os
import cv2
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import numpy as np

Load pre-trained VGG16 model without the top (fully connected) layers

In [2]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

Preprocess input images and extract features

In [3]:
path="/archive/train"
def extract_features_from_folder(path):
    features = []
    for filename in os.listdir(path):
        img_path = os.path.join(path, filename)
        img = image.load_img(img_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        feature = base_model.predict(x).flatten()
        features.append(feature)
    return np.array(features)

In [4]:
# Function to load images and labels from folders
def load_data_from_folders(feature_folders):
    X = []
    y = []
    for label, folder in enumerate(feature_folders):
        for filename in os.listdir(folder):
            img_path = os.path.join(folder, filename)
            img = image.load_img(img_path, target_size=(224, 224))
            x = image.img_to_array(img)
            x = preprocess_input(x)
            X.append(x)
            y.append(label)
    return np.array(X), np.array(y)

In [5]:
feature_folders = ['archive/train/0', 'archive/train/1', 'archive/train/2','archive/train/3', 'archive/train/4', 'archive/train/5', 'archive/train/6']
all_features = []
for folder in feature_folders:
    folder_features = extract_features_from_folder(folder)
    all_features.append(folder_features)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 353ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [6]:
X = np.concatenate(all_features)
y = np.array([0, 1, 2, 3, 4, 5,6])  # Assuming labels for each folder

In [7]:
print(X.shape)
print(y.shape)

(6430, 25088)
(7,)


In [11]:
X, y = load_data_from_folders(feature_folders)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# Reshape X_train and X_test to 2D arrays
n_samples_train, height, width, channels = X_train.shape
n_samples_test = X_test.shape[0]
X_train_flattened = X_train.reshape(n_samples_train, height * width * channels)
X_test_flattened = X_test.reshape(n_samples_test, height * width * channels)

In [17]:
n_samples_train, height, width, channels = X_train.shape
X_train_flattened = X_train.reshape(n_samples_train, height * width * channels)

In [18]:
# Normalize the feature data
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train_flattened)
X_test_normalized = scaler.transform(X_test_flattened)

In [19]:
# Initialize and train a decision tree classifier
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train_flattened, y_train)

DecisionTreeClassifier()

In [ ]:
# Reshape X_test to 2D array
n_samples_test, height, width, channels = X_test.shape
X_test_flattened = X_test.reshape(n_samples_test, height * width * channels)

# Predict emotions for the test set
y_pred = decision_tree.predict(X_test_flattened)

In [ ]:
# Evaluate the performance of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
# Predict emotions for the test set
y_pred = decision_tree.predict(X_test_normalized)

# Evaluate the performance of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Return the trained decision tree classifier
decision_tree

In [ ]:
cap = cv2.VideoCapture(0)

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
# Initialize the emotion labels

In [ ]:
emotion_labels = ['label1', 'label2', 'label3', 'label4', 'label5', 'label6']

In [ ]:
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # Convert frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # Process each detected face
    for (x, y, w, h) in faces:
        # Extract the face region from the frame
        face_roi = frame[y:y+h, x:x+w]
        
        # Preprocess the face region
        face_img = cv2.resize(face_roi, (224, 224))  # Resize to match input size expected by the model
        face_img = image.img_to_array(face_img)
        face_img = preprocess_input(face_img)
        
        # Reshape and normalize the face image
        face_flattened = face_img.reshape(1, -1)
        scaler = StandardScaler()
        face_normalized = scaler.fit_transform(face_flattened)
        
        # Predict emotion label using the decision tree model
        emotion_label = emotion_labels[decision_tree.predict(face_normalized)[0]]
        
        # Display the emotion label on the frame
        cv2.putText(frame, emotion_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
        
        # Draw a rectangle around the face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
    
    # Display the resulting frame
    cv2.imshow('Real-time Emotion Detection', frame)
    
    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()